In [1]:
# Python standard libraries
import math

# Packages for computation and modelling
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.stats import norm
import mne
import pickle

# Packages for visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Self-defined packages
from swlda import SWLDA
import utils

# Magic command to reload packages whenever we run any later cells
%load_ext autoreload
%autoreload 2

In [2]:
BOARD = [["A",    "B",  "C",   "D",    "E",    "F",     "G",    "H"    ],
         ["I",    "J",  "K",   "L",    "M",    "N",     "O",    "P"    ],
         ["Q",    "R",  "S",   "T",    "U",    "V",     "W",    "X"    ],
         ["Y",    "Z",  "Sp",  "1",    "2",    "3",     "4",    "5"    ],
         ["6",    "7",  "8",   "9",    "0",    "Prd",   "Ret",  "Bs"   ],
         ["?",    ",",  ";",   "\\",   "/",    "+",     "-",    "Alt"  ],
         ["Ctrl", "=",  "Del", "Home", "UpAw", "End",   "PgUp", "Shft" ],
         ["Save", "'",  "F2",  "LfAw", "DnAw", "RtAw",  "PgDn", "Pause"],
         ["Caps", "F5", "Tab", "EC",   "Esc",  "email", "!",    "Sleep"]]
BOARD  = np.array(BOARD)
N_ROWS = BOARD.shape[0]  # number of rows
N_COLS = BOARD.shape[1]  # number of columns
M = N_ROWS * N_COLS      # the number of chars on the board

In [3]:
paradigm       = 'RC'  # display paradigm ('RC', 'CB', or 'RD')
NUM_TIMESTAMPS = 195   # number of timestamps in each window to record signals
EPOCH_SIZE     = 15    # required number of features in every epoch
CORE_CHANNELS  = ('EEG_Fz', 'EEG_Cz',  'EEG_P3',  'EEG_Pz',
                  'EEG_P4', 'EEG_PO7', 'EEG_PO8', 'EEG_Oz')
NUM_CORE_CHANNELS  = len(CORE_CHANNELS)  # number of core eletrodes
NUM_TRAIN_WORDS = 5 # number of training words for one participant
NUM_TEST_WORDS  = 5 # number of testing words for one participant

obj = 5 # the index of experiment object (participant)
obj = str(obj) if obj >= 10 else '0'+str(obj)
directory = '/Users/zionshane/Desktop/Duke/Research/BCI_data/EDFData-StudyA'
obj_directory = directory + f'/A{obj}/SE001'

In [5]:
with open(f'./model/A05-model.pkl', 'rb') as f:
    clf = pickle.load(f)

In [13]:
def count_num_same(list_1, list_2):
    if (len(list_1) != len(list_2)):
        raise ValueError('The two input lists should have the same length!')
    num_same = 0
    for i in range(len(list_1)):
        if (list_1[i] == list_2[i]):
            num_same += 1

    return num_same

In [15]:
test_file_indices = [6, 7, 8, 9, 10]
for file_index in test_file_indices:
    file_index = str(file_index) if file_index >= 10 else '0'+str(file_index)
    test_file = directory+f'/A05/SE001/Test/RC/A05_SE001RC_Test{file_index}.edf'
    raw_data = mne.io.read_raw_edf(test_file, preload=True, verbose=False)

    stim_events = mne.find_events(raw=raw_data,
                                stim_channel='StimulusBegin',
                                verbose=False)
    eeg_channels = mne.pick_channels_regexp(raw_data.info['ch_names'], 'EEG')
    raw_data.notch_filter(freqs=60, picks=eeg_channels, verbose=False)
    test_epochs = utils.get_core_epochs(raw_data)

    current_target_events = mne.find_events(raw_data, stim_channel='CurrentTarget',
                                            verbose=False)
    current_target = current_target_events[:,2]
    truth = utils.eventIDs_to_sequence(BOARD, current_target)
    str_truth = ''.join(truth)
    print(f'In test #{file_index}, Participant #{obj} wants to type "{str_truth}".')

    selected_target_events = mne.find_events(raw_data, stim_channel='SelectedTarget',
                                             verbose=False)
    selected_target = selected_target_events[:,2]
    selection = utils.eventIDs_to_sequence(BOARD, selected_target)
    str_selection = ''.join(selection)
    print(f'The speller ends up with "{str_selection}".')

    num_correct = count_num_same(truth, selection)
    print(f'{num_correct} out of {len(truth)} selection is correct.')

In test #06, Participant #05 wants to type "DRIVING".
The speller ends up with "DJ6VING".
5 out of 7 selection is correct.
In test #07, Participant #05 wants to type "QUICKLY".
The speller ends up with "YUICJLY".
5 out of 7 selection is correct.
In test #08, Participant #05 wants to type "TOWARDS".
The speller ends up with "TOWARDS".
7 out of 7 selection is correct.
In test #09, Participant #05 wants to type "DAYLIGHT".
The speller ends up with "DQYKIOG\".
3 out of 8 selection is correct.
In test #10, Participant #05 wants to type "5149736".
The speller ends up with "51Ret9,PrdCtrl".
3 out of 7 selection is correct.


<div style="text-align:justify; width: 97%">

The overall accuracy of Participant #5 is $64\%$. By printing out each word the participant types, we can see that the first three words are actually very accurate whereas the last two words are far from the expectation given that the participant's classifier has a pretty good performance (AUC = $0.82$). The last two severely drag down the average accuracy. However, since the first three are accurate, I tend to believe the lower accuracy in the 4th and 5th word is just caused by randomness.

</div>